# 3.4 Actividad extra: PyTorch en un contexto biológico

En esta actividad extra se propone el uso del paquete PyTorch para acelerar en GPU un problema habitual en análisis de datos biológicos.

En muchos experimentos de citometría de flujo o análisis celular, cada célula puede describirse mediante un vector de características (marcadores, proteínas, etc.). Un análisis común consiste en calcular la distancia euclídea entre un conjunto grande de células y un perfil de referencia, por ejemplo para identificar células similares a un fenotipo dado.

Se pide:

- Partiendo de un conjunto de datos sintéticos que representen perfiles celulares, implementar en NumPy el cálculo de la distancia euclídea entre cada célula y un perfil de referencia.

- Implementar la misma operación utilizando PyTorch, ejecutando el cálculo en GPU.

- Comparar los tiempos de ejecución en CPU y GPU.

- Ejecutar el notebook mediante SLURM, utilizando la cola bohr, y permitir que el número de células N se pase por línea de comandos.

- Analizar los resultados obtenidos y discutir el beneficio del uso de GPU en este tipo de aplicaciones biológicas.


In [ ]:
# imports y argumentos

import sys
import numpy as np
import torch

# Valores por defecto
N = 5_000_000
D = 32

# Leer argumentos si vienen por línea de comandos
if len(sys.argv) > 1:
    N = int(sys.argv[1])
if len(sys.argv) > 2:
    D = int(sys.argv[2])

print("N =", N)
print("D =", D)



In [ ]:
# Datos biológicos simulados

# Perfiles celulares
cells_np = np.random.rand(N, D).astype(np.float32)

# Perfil de referencia (por ejemplo, célula tipo)
ref_np = np.random.rand(D).astype(np.float32)


In [ ]:
# CON NUMPY

def distances_numpy(cells, ref):
    diff = cells - ref
    dist = np.sqrt(np.sum(diff**2, axis=1))
    return dist
    
print("Cálculo de distancias en CPU (NumPy)")
%timeit -r3 distances_numpy(cells_np, ref_np)


In [ ]:
# PYTORCH EN GPU

device = torch.device("cuda")

cells_t = torch.from_numpy(cells_np).to(device)
ref_t = torch.from_numpy(ref_np).to(device)

def distances_torch(cells, ref):
    diff = cells - ref
    dist = torch.sqrt(torch.sum(diff**2, dim=1))
    return dist

# Calentamiento
_ = distances_torch(cells_t, ref_t)

# Sincronización explícita
torch.cuda.synchronize()
print("Cálculo de distancias en GPU (PyTorch)")
%timeit -r3 distances_torch(cells_t, ref_t)


In [ ]:
# COMPROBACIÓN 

dist_cpu = distances_numpy(cells_np[:10], ref_np)
dist_gpu = distances_torch(cells_t[:10], ref_t).cpu().numpy()

print("Ejemplo de distancias (CPU):", dist_cpu)
print("Ejemplo de distancias (GPU, PyTorch):", dist_gpu)


## ANÁLISIS DE LOS RESULTADOS

En esta actividad se simula un problema típico de análisis de datos biológicos, donde se calcula la distancia de un gran número de perfiles celulares respecto a un perfil de referencia. 

En este ejercicio, cada célula se representa mediante un vector de características biológicas (por ejemplo, niveles de expresión génica, intensidades de marcadores proteicos o rasgos fenotípicos), y se compara con un perfil de referencia que puede interpretarse como una célula tipo o un estado basal.

La distancia euclídea entre una célula y el perfil de referencia mide cuán similares son ambos perfiles. Se compara con un grupo control o con la media / percentiles:

- Distancia cercanas a la media → la célula es muy similar al perfil de referencia

- Distancia muy altas → la célula es diferente, lo que puede indicar un estado alterado o distinto

Este tipo de cálculo es habitual en biología computacional y bioinformática, y se utiliza en múltiples contextos, entre ellos: clasificación celular, asignando células a tipos conocidos; detección de estados alterados, como activación, diferenciación o respuesta a un tratamiento; medición de similitud fenotípica entre células individuales; clustering de células, para identificar subpoblaciones; o análisis de poblaciones celulares, especialmente en datos de single-cell (scRNA-seq, citometría, etc.).

En este ejemplo, se ha calculado la distancia euclídea entre un perfil celular de referencia y una población de N = 5·10⁶ células descritas por D = 32 características. La implementación en CPU mediante NumPy presenta un tiempo de ejecución de aproximadamente 248 ms, mientras que la versión en GPU utilizando PyTorch reduce el tiempo a unos 7.4 ms, lo que supone una aceleración de más de un orden de magnitud.

Los valores de distancia obtenidos son equivalentes en ambas implementaciones, lo que confirma la corrección del cálculo.

En definitiva, el uso de GPU mediante PyTorch permite acelerar este tipo de cálculos cuando se trabaja con millones de células y decenas o cientos de características por célula, una situación habitual en estudios biológicos modernos.
El uso de PyTorch permite acelerar significativamente este tipo de cálculos mediante el uso de GPU, manteniendo un código simple en Python.